# Eigenmode simulation of single qubit with readout resonator

In [1]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [2]:
import names as n
import design as d
from qdesignoptimizer.utils.chip_generation import create_chip_base

design, gui = create_chip_base(n.CHIP_NAME, d.chip_type, open_gui=True)
d.render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [3]:
import mini_studies as ms
import optimization_targets as ot

In [4]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: d.render_qiskit_metal_design(design, gui)


opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
    use_simple_resonator_qubit_chi_relation=True,
)

## Creating design analysis objects

In [5]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from design import CoupledLineTee_mesh_names
from qdesignoptimizer.design_analysis_types import MeshingMap

from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [6]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

# map for finer meshing
meshing_map = [
    MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
]

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=meshing_map,
)

[INFO|2025-07-14 16:03:37]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:03PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:03PM [load_ansys_project]: 	Opened Ansys App
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project97
INFO 04:03PM [connect_design]: No active design found (or error getting active design).
INFO 04:03PM [connect]: 	 Connected to project "Project97". No design detected
04:03PM 45s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_res was not in active project.  Designs in acti

## Optimization step

In [7]:
# number of runs of optimization and number of passes for simulation at each run
nbr_iterations = 1
group_passes = 3
delta_f = 0.001
for i in range(nbr_iterations):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-07-14 16:03:46]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-100um",
    "design_var_cl_pos_y_qubit_1": "-300um",
    "design_var_cl_pos_x_qubit_2": "-100um",
    "design_var_cl_pos_y_qubit_2": "300um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "400um",
    "design_var_coupl_length_resonator_2_tee": "160um"
}


pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 17508.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: No project is defined. Project Project97 exists and has been read.
pyaedt INFO: Active Design set to get_mini_study_qb_res
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded
pyaedt INFO: Union of 10 objects has been executed.
sheet_thickness 0.0002
pyaedt INFO: Union of 10 objects has been executed.


INFO 04:04PM [analyze]: Analyzing setup Setup


interface substrate_air
Assigning {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1} to main_Section1 with properties {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1}
interface metal_substrate
Assigning {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1} to readout_wire_name_qubit_1_2 with properties {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1}
interface underside_surface
Assigning {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1} to main_Section2 with properties {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1}
interface metal_air
Assigning {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1} to readout_wire_name_qubit_1_ with properties {'eps_r': 11.4, 'th': 1e-06, 'tan_delta_surf': 1}


KeyboardInterrupt: 

In [ ]:
design_analysis.pratio

{'substrate_air': {'2': Qsurf_main_Section1    8.939671e-07
  dtype: float64},
 'metal_substrate': {'2': Qsurf_readout_wire_name_qubit_1_2    1.918400e-08
  dtype: float64},
 'underside_surface': {'2': Qsurf_main_Section2    6.327069e-11
  dtype: float64},
 'metal_air': {'2': Qsurf_readout_wire_name_qubit_1_    4.300215e-08
  dtype: float64},
 'Junction(inductive energy)': {'2': 0.001198706441869977},
 'dielectric': {'2': 0.9075539941229287}}

## Results

### Eigenmodes

In [ ]:
design_analysis.get_eigenmode_results()

Freq. (GHz)  Quality Factor  Kappas (kHz)    Freq. (Hz)  \
variation mode                                                            
0         0        4.662363    3.541408e+08      0.013165  4.662363e+09   
          1        6.996461    2.281050e+05     30.672102  6.996461e+09   

                 Kappas (Hz)  
variation mode                
0         0        13.165280  
          1     30672.102286

### Cross Kerr

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

0         1
variation                        
0         0  216.455739  0.604541
          1    0.604541  0.000468

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

[INFO|2025-07-14 16:00:02]: Overwritten parameters
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-100um",
    "design_var_cl_pos_y_qubit_1": "-300um",
    "design_var_cl_pos_x_qubit_2": "-100um",
    "design_var_cl_pos_y_qubit_2": "300um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "400um",
    "design_var_coupl_length_resonator_2_tee": "160um"
}

## Close

In [ ]:
# close_ansys()